In [186]:
# Relative imports
import pandas as pd
import numpy as np
import math

In [177]:
raw_df = pd.read_csv("../data/raw/eurovision.csv", index_col=0)

In [178]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2005 entries, 1 to 2005
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   event           2005 non-null   object 
 1   host_city       2005 non-null   object 
 2   year            2005 non-null   int64  
 3   host_country    2005 non-null   object 
 4   event_url       2005 non-null   object 
 5   section         2005 non-null   object 
 6   artist          2005 non-null   object 
 7   song            2004 non-null   object 
 8   artist_url      2005 non-null   object 
 9   image_url       959 non-null    object 
 10  artist_country  2005 non-null   object 
 11  country_emoji   2005 non-null   object 
 12  running_order   2000 non-null   float64
 13  total_points    1913 non-null   float64
 14  rank            1964 non-null   float64
 15  rank_ordinal    1964 non-null   object 
 16  qualified       2005 non-null   bool   
 17  winner          2005 non-null   b

In [179]:
trimmed_df[trimmed_df['year'] == 1959]

,event,host_city,year,host_country,section,artist_country,running_order,total_points,rank,rank_ordinal,qualified,winner
1961,Cannes 1959,Cannes,1959,France,final,Belgium,11.0,9.0,6.0,6th,False,False
1962,Cannes 1959,Cannes,1959,France,final,United Kingdom,10.0,16.0,2.0,2nd,False,False
1963,Cannes 1959,Cannes,1959,France,final,Austria,9.0,4.0,9.0,9th,False,False
1964,Cannes 1959,Cannes,1959,France,final,Switzerland,8.0,14.0,4.0,4th,False,False
1965,Cannes 1959,Cannes,1959,France,final,Sweden,7.0,4.0,9.0,9th,False,False
1966,Cannes 1959,Cannes,1959,France,final,Germany,6.0,5.0,8.0,8th,False,False
1967,Cannes 1959,Cannes,1959,France,final,Netherlands,5.0,21.0,1.0,1st,False,True
1968,Cannes 1959,Cannes,1959,France,final,Monaco,4.0,1.0,11.0,11th,False,False
1969,Cannes 1959,Cannes,1959,France,final,France,1.0,15.0,3.0,3rd,False,False
1970,Cannes 1959,Cannes,1959,France,final,Denmark,2.0,12.0,5.0,5th,False,False


In [180]:
trimmed_df = raw_df.drop(columns=['event_url', 'artist', 'song', 'artist_url', 'image_url', 'country_emoji'])
trimmed_df.drop(trimmed_df[trimmed_df.year == 1956].index, inplace=True) # Drop 1956 because rank is not proper in this year
trimmed_df.drop(trimmed_df[trimmed_df.year == 2020].index, inplace=True) # Drop 2020 because covid
trimmed_df

,event,host_city,year,host_country,section,artist_country,running_order,total_points,rank,rank_ordinal,qualified,winner
1,Turin 2022,Turin,2022,Italy,first-semi-final,Ukraine,6.0,337.0,1.0,1st,True,False
2,Turin 2022,Turin,2022,Italy,first-semi-final,Netherlands,8.0,221.0,2.0,2nd,True,False
3,Turin 2022,Turin,2022,Italy,first-semi-final,Greece,15.0,211.0,3.0,3rd,True,False
4,Turin 2022,Turin,2022,Italy,first-semi-final,Portugal,10.0,208.0,4.0,4th,True,False
5,Turin 2022,Turin,2022,Italy,first-semi-final,Bulgaria,7.0,29.0,16.0,16th,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
1987,Frankfurt 1957,Frankfurt,1957,Germany,final,Austria,5.0,3.0,10.0,10th,False,False
1988,Frankfurt 1957,Frankfurt,1957,Germany,final,Italy,4.0,7.0,6.0,6th,False,False
1989,Frankfurt 1957,Frankfurt,1957,Germany,final,United Kingdom,3.0,6.0,7.0,7th,False,False
1990,Frankfurt 1957,Frankfurt,1957,Germany,final,Luxembourg,2.0,8.0,4.0,4th,False,False


In [189]:
prepped_df = trimmed_df.copy()
prepped_df['winner'] = np.where(prepped_df['winner'] == True, 1, 0)
prepped_df['qualified'] = np.where(prepped_df['qualified'] == True, 1, 0)

#Helper Columns
prepped_df['event-section'] = prepped_df['event'] + prepped_df['section']
contestants_per_contest = prepped_df['event-section'].value_counts().sort_index()
prepped_df['section_contestants'] = prepped_df['event-section'].apply(lambda x: contestants_per_contest.loc[x])

#Prepped Columns - Features to Test
prepped_df['relative_order'] = prepped_df['running_order'] / prepped_df['section_contestants']
prepped_df['first_to_perform'] = np.where(prepped_df['running_order'] == 1, 1, 0)
prepped_df['last_to_perform'] = np.where(prepped_df['running_order'] == prepped_df['section_contestants'], 1, 0)
prepped_df['is_host_country'] = np.where(prepped_df['host_country'] == prepped_df['artist_country'], 1, 0)

#Prepped Columns - Targets to Test
prepped_df['relative_rank'] = (prepped_df['section_contestants'] - prepped_df['rank'] + 1) / prepped_df['section_contestants']
prepped_df['rank_quintiles'] = prepped_df['relative_rank'].apply(lambda x: 6 - math.ceil(5 * x))

In [190]:
prepped_df

,event,host_city,year,host_country,section,artist_country,running_order,total_points,rank,rank_ordinal,qualified,winner,event-section,section_contestants,relative_order,first_to_perform,last_to_perform,is_host_country,relative_rank,rank_quintiles
1,Turin 2022,Turin,2022,Italy,first-semi-final,Ukraine,6.0,337.0,1.0,1st,1,0,Turin 2022first-semi-final,17,0.352941,0,0,0,1.000000,1
2,Turin 2022,Turin,2022,Italy,first-semi-final,Netherlands,8.0,221.0,2.0,2nd,1,0,Turin 2022first-semi-final,17,0.470588,0,0,0,0.941176,1
3,Turin 2022,Turin,2022,Italy,first-semi-final,Greece,15.0,211.0,3.0,3rd,1,0,Turin 2022first-semi-final,17,0.882353,0,0,0,0.882353,1
4,Turin 2022,Turin,2022,Italy,first-semi-final,Portugal,10.0,208.0,4.0,4th,1,0,Turin 2022first-semi-final,17,0.588235,0,0,0,0.823529,1
5,Turin 2022,Turin,2022,Italy,first-semi-final,Bulgaria,7.0,29.0,16.0,16th,0,0,Turin 2022first-semi-final,17,0.411765,0,0,0,0.117647,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1987,Frankfurt 1957,Frankfurt,1957,Germany,final,Austria,5.0,3.0,10.0,10th,0,0,Frankfurt 1957final,10,0.500000,0,0,0,0.100000,5
1988,Frankfurt 1957,Frankfurt,1957,Germany,final,Italy,4.0,7.0,6.0,6th,0,0,Frankfurt 1957final,10,0.400000,0,0,0,0.500000,3
1989,Frankfurt 1957,Frankfurt,1957,Germany,final,United Kingdom,3.0,6.0,7.0,7th,0,0,Frankfurt 1957final,10,0.300000,0,0,0,0.400000,4
1990,Frankfurt 1957,Frankfurt,1957,Germany,final,Luxembourg,2.0,8.0,4.0,4th,0,0,Frankfurt 1957final,10,0.200000,0,0,0,0.700000,2


In [194]:
prepped_df.to_csv('../data/preprocessed/eurovision_data_preprocessed.csv')